In [4]:
import pandas as pd

train_trans = pd.read_csv("D:/project/data/train_transaction.csv")
train_id = pd.read_csv("D:/project/data/train_identity.csv")
test_trans = pd.read_csv("D:/project/data/test_transaction.csv")
test_id = pd.read_csv("D:/project/data/test_identity.csv")

#merge to 1
test_df = test_trans.merge(test_id,on="TransactionID",how="left")
train_df=train_trans.merge(train_id,on="TransactionID",how="left")




# Dataset Cleaning and Preprocessing

This section describes the data preprocessing pipeline applied to the IEEE-CIS Fraud Detection dataset, including the following steps:

- Remove the label column **`isFraud`** from the feature set.
- Identify and handle missing values.
- Encode categorical features into numerical representations.
- Ensure that training and testing datasets share the same feature columns.
- Split the dataset into training and validation sets based on transaction time.


In [ ]:
y_train = train_df["isFraud"]
X_train = train_df.drop(columns=["isFraud"])
X_test  = test_df.copy()

#  Align columns
X_train, X_test = X_train.align(X_test, join="outer", axis=1)

#  Concat to take cat_cols/num_cols
X_all = pd.concat([X_train, X_test], axis=0)

cat_cols = X_all.select_dtypes(include=["object"]).columns.tolist()
num_cols = X_all.select_dtypes(exclude=["object"]).columns.tolist()

# Fill missing 
for col in num_cols:
    median = X_train[col].median()
    X_train[col] = X_train[col].fillna(median)
    X_test[col]  = X_test[col].fillna(median)

for col in cat_cols:
    X_train[col] = X_train[col].fillna("missing")
    X_test[col]  = X_test[col].fillna("missing")

#  Encode train+test 
from sklearn.preprocessing import LabelEncoder

X_all = pd.concat([X_train, X_test], axis=0)
encoders = {}

for col in cat_cols:
    le = LabelEncoder()
    X_all[col] = le.fit_transform(X_all[col].astype(str))
    encoders[col] = le

# split
X_train = X_all.iloc[:len(X_train)]
X_test  = X_all.iloc[len(X_train):]

#  Tensor
import torch
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor  = torch.tensor(X_test.values, dtype=torch.float32)


c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keep

In [7]:
X_train_out = X_train.copy()
X_train_out["isFraud"] = y_train.values

X_train_out.to_csv("train_processed.csv", index=False)
X_test.to_csv("test_processed.csv", index=False)

print("Saved train_processed.csv & test_processed.csv")


Saved train_processed.csv & test_processed.csv
